In [1]:
%load_ext autoreload
%autoreload 2
import pygsti
from pygsti.modelpacks import smq1Q_XYI as mp
from pygsti.tools.leakage import leaky_qubit_model_from_pspec, construct_leakage_report, add_lago_models, gate_leakage_profile
from pygsti.report import construct_standard_report
from pygsti.tools.optools import unitary_to_superop
from pygsti.data import simulate_data
from pygsti.protocols import StandardGST, ProtocolData, GateSetTomography, GSTGaugeOptSuite, GSTBadFitOptions
import numpy as np
import scipy.linalg as la

## GST: modeling a leaky qubit as a qutrit

This short notebook shows how (data from) an experiment design for a two-level system can be used to fit a three-level sytem model, and how to generate a special report to provide insights for these models. The report includes special gate error metrics that reflect the distinguished role of the first two levels in the three-level system.

In [2]:
def with_leaky_gate(m, gate_label, strength):
    rng = np.random.default_rng(0)
    v = np.concatenate([[0.0], rng.standard_normal(size=(2,))])
    v /= la.norm(v)
    H = v.reshape((-1, 1)) @ v.reshape((1, -1))
    H *= strength
    U = la.expm(1j*H)
    m_copy = m.copy()
    G_ideal = m_copy.operations[gate_label]
    from pygsti.modelmembers.operations import ComposedOp, StaticUnitaryOp
    m_copy.operations[gate_label] = ComposedOp([G_ideal, StaticUnitaryOp(U, basis=m.basis)])
    return m_copy, v


In [3]:
ed = mp.create_gst_experiment_design(max_max_length=16)
ps = mp.processor_spec()
tm3 = leaky_qubit_model_from_pspec(ps, mx_basis='l2p1')
tm3.convert_members_inplace('CPTPLND')
# ^ We could use basis = 'gm' instead of 'l2p1'. We prefer 'l2p1'
#   because it makes process matrices easier to interpret in leakage
#   modeling.
se = 'multinomial'
leaky_gate = ('Gxpi2', 0)
dm3, leaking_state = with_leaky_gate(tm3, leaky_gate, strength=0.125)
if se == 'none':
    import pygsti.objectivefns.objectivefns as pg_ofns
    # Forward simulation can raise errors if there are positive
    # probabilities < pg_ofns.DEFAULT_MIN_PROB_CLIP.
    # We change that default here so forward simulation works with
    # the ideal model.
    pg_ofns.DEFAULT_MIN_PROB_CLIP = 1e-12
ds = simulate_data(dm3, ed.all_circuits_needing_data, num_samples=10_000, sample_error=se, seed=1997)

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/modelmembers/povms/__init__.py:615: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  errgen_vec = _np.linalg.lstsq(phys_directions, soln.x)[0]


In [4]:
gopsuite = GSTGaugeOptSuite.cast('stdgaugeopt')
gopsuite.gaugeopt_target = tm3
bfo = {'actions': ['wildcard1d'], 'wildcard1d_reference': 'diamond distance', 'threshold': 0.0}
bfo = GSTBadFitOptions.cast(bfo)
gst = GateSetTomography(initial_model=dm3, gaugeopt_suite=gopsuite, verbosity=1, name='CPTPLND', badfit_options=bfo)
pd = ProtocolData(ed, ds)
res = gst.run(pd)

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/objectivefns/objectivefns.py:4494: RuntimeWarning: divide by zero encountered in divide
  p5over_lsvec = 0.5/lsvec


--- Iterative GST: [##################################################] 100.0%  616 circuits ---


In [5]:
import copy
resc = copy.deepcopy(res)
add_lago_models(resc, 'CPTPLND', verbosity=1)
est = resc.estimates['CPTPLND']

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/models/model.py:145: UserWarning: Model.num_modeltest_params could not obtain number of *non-gauge* parameters - using total instead
  _warnings.warn(("Model.num_modeltest_params could not obtain number of *non-gauge* parameters"


In [6]:
report_dir = f'example_files/leakage-report-automagic-251016-NleakNone-{se}'
report_object = construct_standard_report(resc, advanced_options={'n_leak': None}, title=f'fidelity-gop sample_error={se}')
# report_object, updated_res = construct_leakage_report(res, title='easy leakage analysis!')
# ^ Each estimate in updated_res has a new gauge-optimized model.
#   The gauge optimization was done to reflect how our target gates
#   are only _really_ defined on the first two levels of our
#   three-level system.
#   
report_object.write_html(report_dir)

Running idle tomography
Computing switchable properties


/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/report/plothelpers.py:367: UserWarning:

Max-model params (92) <= model params (158)!  Using k == 1.

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/forwardsims/mapforwardsim.py:732: UserWarning:

Generating dense process matrix representations of circuits or gates 
can be inefficient and should be avoided for the purposes of forward 
simulation/calculation of circuit outcome probability distributions 
when using the MapForwardSimulator.

/Users/rjmurr/Documents/pygsti-leakage/hrl-repo/pygsti_repo/pygsti/report/plothelpers.py:367: UserWarning:

Max-model params (92) <= model params (158)!  Using k == 1.

